## Proyecto 3 - Modelación
**Integrantes:**
- Ruth de León, 22428 
- Héctor Penedo, 22217 
- Rodrigo Mansilla, 22611



## Variable Respuesta


Se selecciona como variable de respuesta Caudef, que corresponde al código CIE-10 de la causa de defunción. Esta variable es categórica multinivel –cada código CIE-10 representa una clase distinta– y refleja directamente el desenlace de interés, permitiendo analizar la asociación entre cada causa de muerte y factores socioeconómicos o geográficos.

**Partición del conjunto de datos**

Con 1 008 732 registros y 66 variables, se implementa un muestreo estratificado sobre Caudef: 70 % de los casos se destina al conjunto de entrenamiento y 30 % al de prueba. La distribución de clases presenta una cola larga: las cinco causas más frecuentes (I219: 7,13 %; J189: 5,41 %; E149: 3,67 %; R98X: 3,29 %; X599: 3,12 %) suman apenas el 22,6 % del total, mientras que el 77,4 % restante se reparte en miles de códigos con muy baja frecuencia, evidenciando un desequilibrio severo de clases.

**Limpieza y reducción de dimensionalidad**
Para abordar la alta dimensionalidad y los valores erróneos:

- Se marcan como NA los valores “999”, “9999” o negativos en variables numéricas (edad, año, etc.).

- Se imputan las variables numéricas con la mediana y las categóricas con la moda.

- Se eliminan aquellas variables con más del 80 % de valores faltantes o con varianza prácticamente nula.

**Codificación y agrupamiento**

- Las variables numéricas se normalizan (centrado y escala).

- Las variables categóricas se transforman mediante one-hot encoding.

- Se agrupan las categorías de baja frecuencia (por ejemplo, municipios con menos del 1 % de observaciones) bajo la etiqueta “Otros” para evitar alta cardinalidad.

## Preprocesamiento de datos


In [18]:
# 0. IMPORTS Y RUTAS
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

BASE_DIR = r"C:\Users\rodri\Documents\Data-Mining\Proyecto-3-EDA-Clustering"
DATA_DIR = os.path.join(BASE_DIR, "data")


### Carga y Eliminación de variables con observaciones nulas


In [19]:
# 1. CARGAR DATOS
df = pd.read_csv(os.path.join(DATA_DIR, "master.csv"))
X_raw = df.drop(columns="Caudef")
y_raw = df["Caudef"].astype(str)   # homogéneo como string
# 2. AGRUPAR CLASES RARAS EN y
min_count = 2
vc = y_raw.value_counts()
raras = vc[vc < min_count].index
y = y_raw.replace(raras, "Otros")


C:\Users\rodri\AppData\Local\Temp\ipykernel_27780\1242429795.py:2: DtypeWarning: Columns (15,29,33,34,35,52,53,54,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(DATA_DIR, "master.csv"))


### Partición estratificada 70 / 30 sobre `Caudef`

In [20]:
# 3. SPLIT ESTRATIFICADO 70/30
RANDOM_STATE = 123
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y,
    test_size=0.30,
    stratify=y,
    random_state=RANDOM_STATE
)
# 4. IDENTIFICAR VARIABLES
numeric_features = X_train.select_dtypes(include=['number']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object','category']).columns.tolist()

# 4.1 Forzar todas las categóricas a str
for col in categorical_features:
    X_train[col] = X_train[col].astype(str)
    X_test [col] = X_test [col].astype(str)
# 5. AGRUPAR NIVELES CATEGÓRICOS RAROS (<1% en TRAIN) EN "Otros"
threshold = 0.01
for col in categorical_features:
    freqs = X_train[col].value_counts(normalize=True)
    rares = freqs[freqs < threshold].index
    X_train[col] = X_train[col].where(~X_train[col].isin(rares), other="Otros")
    X_test [col] = X_test [col].where(~X_test [col].isin(freqs.index), other="Otros")


### Definición de pipelines de preprocesamiento

In [21]:
# 6. PIPELINES: IMPUTACIÓN, ESCALADO Y DUMMIES
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler' , StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot' , OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numeric_features),
    ('cat', cat_pipeline, categorical_features)
])


### Ajuste del preprocesador y transformación de train/test

In [22]:
# 7. AJUSTAR Y TRANSFORMAR
X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep  = preprocessor.transform(X_test)

# Reconstruir DataFrames con nombres de columnas
ohe_cols = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
all_cols = numeric_features + list(ohe_cols)
X_train_prep = pd.DataFrame(X_train_prep, columns=all_cols)
X_test_prep  = pd.DataFrame(X_test_prep,  columns=all_cols)

print("Train preprocessed:", X_train_prep.shape)
print("Test  preprocessed:", X_test_prep.shape)


Train preprocessed: (706112, 241)
Test  preprocessed: (302620, 241)


### Guardar a CSV para modelado posterior

In [ ]:
# 8. GUARDAR CSVs
os.makedirs(os.path.join(DATA_DIR, "train"), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, "test"),  exist_ok=True)

X_train_prep.to_csv(os.path.join(DATA_DIR, "train", "X_train.csv"), index=False)
y_train.to_frame(name="Caudef").to_csv(os.path.join(DATA_DIR, "train", "y_train.csv"), index=False)
X_test_prep .to_csv(os.path.join(DATA_DIR, "test" , "X_test.csv"),  index=False)
y_test .to_frame(name="Caudef") .to_csv(os.path.join(DATA_DIR, "test" , "y_test.csv"),  index=False)


## Random Forest


## Metodología de Modelado con Random Forest (Python / Jupyter)

###  Desafíos computacionales y soluciones  
Con ~700 000 observaciones y cientos de variables, entrenar un Random Forest presenta un desafio computacional.  

**Soluciones propuestas**:  
- **Muestreo estratificado**: usar solo un 15–30 % de los datos para el _tuning_ (CV), manteniendo la proporción por `Caudef`.  
- **Paralelización**: con `joblib` o el parámetro `n_jobs=-1` de scikit-learn, aprovechar todos los núcleos de CPU.  
- **Reducción de árboles** en la fase de tuning (p. ej. 150–300) y simplificación de la grilla de hiperparámetros.  
- **Menos pliegues** (5-fold CV) en lugar de 10-fold, equilibrando robustez y tiempo de cómputo.

---

### Trade-offs sacrificados  

| Aspecto                | Pleno rendimiento    | Enfoque actual                  |
|------------------------|----------------------|---------------------------------|
| # árboles tuning       | 500–1 000            | ~200                            |
| Búsqueda hiperparáms   | grillas amplías      | grillas reducidas (2–3 valores) |
| Muestra tuning         | 100 % train set      | 15–30 % de train set            |
| CV folds               | 10                   | 5                               |

_Estos ajustes aceleran el entrenamiento, pero pueden:_  
- Pérdida leve de precisión en la selección de hiperparámetros.  
- Mayor varianza en la estimación CV.  
- Menos estabilidad en la importancia de variables.

In [1]:
from sklearn.ensemble     import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics      import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot   as plt
import numpy               as np
import pandas              as pd
# 9.2 Cargar datos procesados
BASE = r"C:\Users\rodri\Documents\Data-Mining\Proyecto-3-EDA-Clustering\data"
X_train = pd.read_csv(f"{BASE}/train/X_train.csv")
y_train = pd.read_csv(f"{BASE}/train/y_train.csv")["Caudef"]
X_test  = pd.read_csv(f"{BASE}/test/X_test.csv")
y_test  = pd.read_csv(f"{BASE}/test/y_test.csv")["Caudef"]


KeyboardInterrupt: 

### Muestreo Estratificado

In [ ]:
# 9.3 Muestreo estratificado para tuning (15 % de train)
RND = 123
X_tune, _, y_tune, _ = train_test_split(
    X_train, y_train,
    train_size    = 0.15,
    stratify      = y_train,
    random_state  = RND
)


### Control de Entrenamiento

In [ ]:
# 9.4 Definir CV y número reducido de árboles
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RND)
base_trees = 200


### Modelo 1: Base

In [ ]:
p      = X_tune.shape[1]  
model1 = RandomForestClassifier(  
    n_estimators     = base_trees,  
    max_features     = "sqrt",  
    min_samples_leaf = 1,  
    n_jobs           = -1,  
    random_state     = RND  
)  
scores1 = cross_val_score(model1, X_tune, y_tune, cv=cv, scoring="accuracy")  
print("Modelo 1 CV Accuracy:", np.round(scores1,3), "→ mean =", np.round(scores1.mean(),3))  


### Evaluación

In [ ]:
model1.fit(X_tune, y_tune)  
y_pred1  = model1.predict(X_test)  
y_proba1 = model1.predict_proba(X_test)  

print("Confusion Matrix Modelo 1:\n", confusion_matrix(y_test, y_pred1))  
print(classification_report(y_test, y_pred1, zero_division=0))  

plt.figure(figsize=(8,6))  
for i, cls in enumerate(model1.classes_):  
    fpr, tpr, _ = roc_curve((y_test==cls).astype(int), y_proba1[:,i])  
    plt.plot(fpr, tpr, label=f"{cls} (AUC={auc(fpr,tpr):.2f})")  
plt.plot([0,1],[0,1],"--",color="grey")  
plt.title("ROC Modelo 1"); plt.legend(loc="lower right"); plt.show()  


###  Modelo 2: Tuning `max_features` 

In [ ]:
param_grid2 = {"max_features": ["sqrt","log2"]}  
gs2 = GridSearchCV(  
    RandomForestClassifier(  
        n_estimators     = base_trees,  
        min_samples_leaf = 1,  
        n_jobs           = -1,  
        random_state     = RND  
    ),  
    param_grid = param_grid2,  
    cv         = cv,  
    scoring    = "accuracy",  
    n_jobs     = -1  
)  
gs2.fit(X_tune, y_tune)  
print("Modelo 2 best params:", gs2.best_params_)  
print("Modelo 2 CV Accuracy:", np.round(gs2.best_score_,3))  


#### Evaluación

In [ ]:
best2    = gs2.best_estimator_  
y_pred2  = best2.predict(X_test)  
y_proba2 = best2.predict_proba(X_test)  

print("Confusion Matrix Modelo 2:\n", confusion_matrix(y_test, y_pred2))  
print(classification_report(y_test, y_pred2, zero_division=0))  

plt.figure(figsize=(8,6))  
for i, cls in enumerate(best2.classes_):  
    fpr, tpr, _ = roc_curve((y_test==cls).astype(int), y_proba2[:,i])  
    plt.plot(fpr, tpr, label=f"{cls} (AUC={auc(fpr,tpr):.2f})")  
plt.plot([0,1],[0,1],"--",color="grey")  
plt.title("ROC Modelo 2"); plt.legend(loc="lower right"); plt.show()  


### Modelo 3: tuning max_features y min_samples_leaf


In [ ]:
param_grid3 = {  
    "max_features"     : ["sqrt","log2"],  
    "min_samples_leaf" : [1,10]  
}  
gs3 = GridSearchCV(  
    RandomForestClassifier(  
        n_estimators = base_trees,  
        n_jobs       = -1,  
        random_state = RND  
    ),  
    param_grid = param_grid3,  
    cv         = cv,  
    scoring    = "accuracy",  
    n_jobs     = -1  
)  
gs3.fit(X_tune, y_tune)  
print("Modelo 3 best params:", gs3.best_params_)  
print("Modelo 3 CV Accuracy:", np.round(gs3.best_score_,3))  


#### Evaluación

In [ ]:
best3    = gs3.best_estimator_  
y_pred3  = best3.predict(X_test)  
y_proba3 = best3.predict_proba(X_test)  

print("Confusion Matrix Modelo 3:\n", confusion_matrix(y_test, y_pred3))  
print(classification_report(y_test, y_pred3, zero_division=0))  

plt.figure(figsize=(8,6))  
for i, cls in enumerate(best3.classes_):  
    fpr, tpr, _ = roc_curve((y_test==cls).astype(int), y_proba3[:,i])  
    plt.plot(fpr, tpr, label=f"{cls} (AUC={auc(fpr,tpr):.2f})")  
plt.plot([0,1],[0,1],"--",color="grey")  
plt.title("ROC Modelo 3"); plt.legend(loc="lower right"); plt.show()  

 ### Comparar distribuciones CV (boxplot)

In [ ]:
# 9.8 Comparar distribuciones CV (boxplot)
results = pd.DataFrame({
    "Base"    : cross_val_score(model1, X_tune, y_tune, cv=cv),
    "Tuning1" : gs2.cv_results_["mean_test_score"],
    "Tuning2" : gs3.cv_results_["mean_test_score"]
})
results.plot.box(grid=True, title="CV Accuracy comparado"); plt.show()


### Modelo Final

In [ ]:
best_params = gs3.best_params_  
final_rf    = RandomForestClassifier(  
    n_estimators     = 400,  
    max_features     = best_params["max_features"],  
    min_samples_leaf = best_params["min_samples_leaf"],  
    n_jobs           = -1,  
    random_state     = RND  
)  
final_rf.fit(X_train, y_train)  

#### Evaluación en test


In [ ]:
y_pred_f  = final_rf.predict(X_test)  
y_proba_f = final_rf.predict_proba(X_test)  

print("Confusion Matrix Modelo Final:\n", confusion_matrix(y_test, y_pred_f))  
print(classification_report(y_test, y_pred_f, zero_division=0))  

plt.figure(figsize=(8,6))  
for i, cls in enumerate(final_rf.classes_):  
    fpr, tpr, _ = roc_curve((y_test==cls).astype(int), y_proba_f[:,i])  
    plt.plot(fpr, tpr, label=f"{cls} (AUC={auc(fpr,tpr):.2f})")  
plt.plot([0,1],[0,1],"--",color="grey")  
plt.title("ROC Modelo Final"); plt.legend(loc="lower right"); plt.show()  

### Importancia de las variables

In [ ]:
importances = pd.Series(final_rf.feature_importances_, index=X_train.columns)  
top20       = importances.nlargest(20)  
top20.sort_values().plot.barh(title="Top 20 Importancias"); plt.show()